In [1]:
import sys
sys.path.append('../scripts/')
from robot import *
from scipy.stats import multivariate_normal
from matplotlib.patches import Ellipse

In [2]:
def sigma_ellipse(p, cov, n): 
    eig_vals, eig_vec = np.linalg.eig(cov)
    ang = math.atan2(eig_vec[:,0][1], eig_vec[:,0][0])/math.pi*180
    return Ellipse(p, width=n*math.sqrt(eig_vals[0]),height=n*math.sqrt(eig_vals[1]), angle=ang, fill=False, color="blue", alpha=0.5)

In [3]:
class KalmanFilter: 
    def __init__(self, envmap, init_pose, motion_noise_stds, distance_dev_rate=0.14, direction_dev=0.05):
        c = np.diag([1e+4, 1e+4, 1e+4]) if init_pose is None else np.diag([1e-10, 1e-10, 1e-10]) #globalかどうかで共分散行列を変える
        self.belief = multivariate_normal(mean=init_pose, cov=c) #cov=cに変更
        self.motion_noise_stds = motion_noise_stds
        self.map = envmap 
        self.distance_dev_rate = distance_dev_rate
        self.direction_dev = direction_dev
        
    def observation_update(self, observation): 
        for d in observation:
            z = d[0]
            obs_id = d[1]
            
            ##Hの計算##
            mx, my = self.map.landmarks[obs_id].pos
            mux, muy, mut = self.belief.mean
            q = (mux - mx)**2 + (muy - my)**2
            sqrtq = np.sqrt(q)
            H = np.array([[(mux - mx)/sqrtq, (muy - my)/sqrtq, 0.0],  [(my - muy)/q, (mux - mx)/q, -1.0]])
            
            ##Qの計算##
            hmu = IdealCamera.relative_polar_pos(self.belief.mean, self.map.landmarks[obs_id].pos)
            distance_dev = self.distance_dev_rate*hmu[0]
            Q = np.diag(np.array([distance_dev**2, self.direction_dev**2]))
            
            ##カルマンゲインの計算##
            K = self.belief.cov.dot(H.T).dot(np.linalg.inv(Q + H.dot(self.belief.cov).dot(H.T)))
            
            ##更新##
            self.belief.mean += K.dot(z - hmu)
            self.belief.cov = (np.eye(3) - K.dot(H)).dot(self.belief.cov)
        
    def motion_update(self, nu, omega, time):
        if abs(omega) < 1e-5: omega = 1e-5 #ゼロにすると式が変わるので避ける
        if abs(nu) < 1e-5:         nu = 1e-5

        v = self.motion_noise_stds
        M = np.diag([v["nn"]**2*abs(nu)/time + v["no"]**2*abs(omega)/time, 
                     v["on"]**2*abs(nu)/time + v["oo"]**2*abs(omega)/time])
        
        t = self.belief.mean[2]
        st, ct = math.sin(t), math.cos(t)
        stw, ctw = math.sin(t + omega*time), math.cos(t + omega*time)
        A = np.array([[(stw - st)/omega,    -nu/(omega**2)*(stw - st) + nu/omega*time*ctw],
                                 [(-ctw + ct)/omega, -nu/(omega**2)*(-ctw + ct) + nu/omega*time*stw],
                                 [0,                                time]] )
        
        F = np.diag([1.0, 1.0, 1.0])
        F[0, 2] = nu / omega * (math.cos(t + omega * time) - math.cos(t))
        F[1, 2] = nu / omega * (math.sin(t + omega * time) - math.sin(t))
            
        self.belief.cov = F.dot(self.belief.cov).dot(F.T) + A.dot(M).dot(A.T)
        self.belief.mean = IdealRobot.state_transition(nu, omega, time, self.belief.mean)
        
    def draw(self, ax, elems):
        ##xy平面上の誤差の3シグマ範囲##
        e = sigma_ellipse(self.belief.mean[0:2], self.belief.cov[0:2, 0:2], 3)
        elems.append(ax.add_patch(e))

        ##θ方向の誤差の3シグマ範囲##
        x, y, c = self.belief.mean
        sigma3 = math.sqrt(self.belief.cov[2, 2])*3
        xs = [x + math.cos(c-sigma3), x, x + math.cos(c+sigma3)]
        ys = [y + math.sin(c-sigma3), y, y + math.sin(c+sigma3)]
        elems += ax.plot(xs, ys, color="blue", alpha=0.5)

In [4]:
class KfAgent(Agent): 
    def __init__(self, time_interval, nu, omega, init_pose, envmap, \
                motion_noise_stds={"nn":0.19, "no":0.001, "on":0.13, "oo":0.2}):
        super().__init__(nu, omega)
        self.kf = KalmanFilter(envmap, init_pose, motion_noise_stds) 
        self.time_interval = time_interval
        
        self.prev_nu = 0.0
        self.prev_omega = 0.0
        
    def decision(self, observation=None):
        self.kf.motion_update(self.prev_nu, self.prev_omega, self.time_interval) 
        self.prev_nu, self.prev_omega = self.nu, self.omega
        self.kf.observation_update(observation) 
        return self.nu, self.omega
        
    def draw(self, ax, elems):
        self.kf.draw(ax, elems)

In [5]:
def trial(animation): ###kfglobalexec
    time_interval = 0.1
    world = World(30, time_interval, debug=not animation)

    ## 地図を生成して3つランドマークを追加 ##
    m = Map()                                  
    m.append_landmark(Landmark(-4,2))
    m.append_landmark(Landmark(2,-3))
    m.append_landmark(Landmark(3,3))
    world.append(m)          

    ## ロボットを作る ##
    particle_pose = np.array([
        np.random.uniform(-5.0, 5.0),
        np.random.uniform(-5.0, 5.0),
        np.random.uniform(-math.pi, math.pi)]
    ).T
    robot_pose = np.array([
        np.random.uniform(-5.0, 5.0),
        np.random.uniform(-5.0, 5.0),
        np.random.uniform(-math.pi, math.pi)]
    ).T
    
    a = KfAgent(time_interval, 0.2, 10.0/180*math.pi, particle_pose, m)    #初期姿勢をparticle_poseに
    r = Robot(robot_pose, sensor=Camera(m), agent=a, color="red")  #初期姿勢をrobot_poseに
    world.append(r)

    world.draw()
 #   world.ani.save('/tmp/anm.gif', writer='imagemagick', fps=1) #描画用    
    return (r.pose, a.kf.belief.mean)

In [7]:
ok = 0 ###kfglobal1exec
for i in range(1000):
    actual, estm = trial(False)
    diff = math.sqrt((actual[0]-estm[0])**2 + (actual[1]-estm[1])**2)
    print(i, "真値:", actual, "推定値", estm, "誤差:", diff)
    if diff <= 1.0:
        ok += 1

ok

0 真値: [-4.79501071  3.15046571  5.14527759] 推定値 [-2.47349447  2.43017676  3.25025411] 誤差: 2.430689991437104
1 真値: [-6.72319574  5.96651641  5.15274346] 推定値 [2.08013694 3.18975358 3.19039784] 誤差: 9.23087634047084
2 真値: [-5.14347923 -3.19135802  6.16572454] 推定値 [0.50994112 2.37121194 8.26083621] 誤差: 7.931162979869448
3 真値: [-4.62303776  2.36019378  7.24053331] 推定値 [-4.07303155  1.22815042  9.07097743] 誤差: 1.2585821402064314
4 真値: [ 0.731242   -1.94679804  6.92177454] 推定値 [ 0.81138859 -2.08251204  0.59068199] 誤差: 0.15761270803197167
5 真値: [-3.03713289  4.47396287  5.50765358] 推定値 [-2.00495429  0.68578681  3.57513616] 誤差: 3.926279470590942
6 真値: [2.90946937 2.5859126  8.6256011 ] 推定値 [ 3.18529329  2.58722894 -3.44678847] 誤差: 0.2758270620800071
7 真値: [1.06555376 0.85203249 4.46963563] 推定値 [1.14111324 1.01334725 4.36503998] 誤差: 0.17813389732220458
8 真値: [-0.71176959 -2.75809271  4.27316522] 推定値 [-0.47699186 -2.7020743   4.28269954] 誤差: 0.24136827983306772
9 真値: [-2.88986139 -3.08134919  3.83

/usr/local/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


19 真値: [0.69506927 1.33446499 7.147172  ] 推定値 [0.91298465 1.30292038 0.93561036] 誤差: 0.22018668734919083
20 真値: [ 6.57522365 -3.77851284  2.16034159] 推定値 [-1.77076616 -0.49693417  4.84654332] 誤差: 8.967959881037434
21 真値: [ 3.30457844 -2.5897519   8.5476523 ] 推定値 [ 1.65160044 -2.04459122  1.89695297] 誤差: 1.7405563643937865
22 真値: [ 0.05915596 -1.79973348  6.18741208] 推定値 [-0.59639147 -2.95770395  6.62762894] 誤差: 1.3306532301603302
23 真値: [ 2.95922194 -1.53568328  5.66582467] 推定値 [ 1.89374932 -1.04973094  6.33157325] 誤差: 1.1710600234100705
24 真値: [ 4.50293633 -0.30256292  7.81564618] 推定値 [ 3.40810591 -0.38991106  1.14939351] 誤差: 1.0983093134924486
25 真値: [5.20200468 5.30567914 3.49781529] 推定値 [ 3.68792066 -0.30645425  1.31592933] 誤差: 5.812786912529578
26 真値: [-2.11001645 -1.6608707   4.64725774] 推定値 [-1.24521255 -0.99511358 11.05470431] 誤差: 1.0913836810635869
27 真値: [ 4.0325665  -0.55609632  3.62634465] 推定値 [3.20791821 0.9084156  3.98804059] 誤差: 1.6807260322412851
28 真値: [1.30805026 4.85

95 真値: [-3.47671567  3.14277774  3.98437585] 推定値 [-2.74221001  2.72245088  9.56535246] 誤差: 0.8462701894007879
96 真値: [4.10402249 0.41881175 8.04164811] 推定値 [3.74213343 0.61140439 7.89968404] 誤差: 0.4099458706726687
97 真値: [-0.1013466  -2.34111782  3.76595698] 推定値 [-0.2639517  -2.32325406 10.0880051 ] 誤差: 0.16358340925857223
98 真値: [ 0.04396747 -3.95053674  6.48371943] 推定値 [-0.28561862 -2.34324023  5.73086897] 誤差: 1.6407404045968328
99 真値: [ 2.34192097 -0.07646438  8.09042321] 推定値 [ 2.48595527 -0.33119402  1.82038384] 誤差: 0.2926312859188402
100 真値: [ 0.27903593 -3.42117959  7.35199701] 推定値 [ 0.67746257 -2.5064675   0.45596422] 誤差: 0.9977183923265852
101 真値: [-0.48766196 -4.31173779  7.34640906] 推定値 [ 0.40368145 -4.8147223   7.68402576] 誤差: 1.0234678813058107
102 真値: [-3.06279426 -3.86925846  6.7111265 ] 推定値 [ 2.53994198  2.8417729  -1.42064551] 誤差: 8.742344951349054
103 真値: [ 3.58386719 -1.42099537  8.21921156] 推定値 [ 3.41115269 -1.75492465  8.19853458] 誤差: 0.37595087740437316
104 真値: [-0

170 真値: [2.19780382 0.89213331 3.59998664] 推定値 [1.02376347 1.97954301 3.23739279] 誤差: 1.6002595403416318
171 真値: [-5.39483484 -4.67607172  5.91285256] 推定値 [ 2.24528782 -0.27662031  1.51865573] 誤差: 8.816271720989693
172 真値: [-6.48452172  1.72312798  5.65722793] 推定値 [-3.37459982  4.63553839  3.68442289] 誤差: 4.2607216088820214
173 真値: [-0.04615022  1.96224469  6.90459509] 推定値 [-0.24096986  1.3247507   6.95598915] 誤差: 0.6665982901643301
174 真値: [-0.37001664  4.28812699  5.46828732] 推定値 [-0.18537252  3.99580907  5.62217502] 誤差: 0.3457502179649893
175 真値: [-6.18662026  3.176977    5.23832056] 推定値 [-1.70365966  2.46543807  2.76935411] 誤差: 4.539077373942096
176 真値: [-0.27522884  0.68777885  7.74957608] 推定値 [-0.31909082  0.35826328  7.78237354] 誤差: 0.3324219954978791
177 真値: [4.33915731 1.97712845 3.48291042] 推定値 [4.29079907 2.18507001 3.50270947] 誤差: 0.21349053800087542
178 真値: [-2.96867383 -0.7055483   8.03341566] 推定値 [-3.452967   -1.09764964  1.53187918] 誤差: 0.6231238558565672
179 真値: [-2.75

...

770 真値: [-1.10959273 -0.97031674  7.51414195] 推定値 [-2.36070915 -1.49307171  7.45542454] 誤差: 1.3559369658929377
771 真値: [-3.60466286 -3.86558156  5.73751299] 推定値 [-0.87271562  3.3835716   4.48010731] 誤差: 7.746854664148657
772 真値: [ 4.95969194 -1.78333597  3.86579742] 推定値 [ 5.28235775 -1.87141286 10.06419106] 誤差: 0.3344708669354715
773 真値: [-0.237625   -2.52094964  5.33144917] 推定値 [-0.6816589  -3.43216446  5.69224359] 誤差: 1.0136461662496037
774 真値: [ 1.29315185 -1.70420682  6.27904581] 推定値 [ 1.32711749 -1.88548947  0.02390945] 誤差: 0.18443715844563172
775 真値: [1.33397753 0.15438015 6.02713448] 推定値 [1.29907862 0.0280515  6.07892703] 誤差: 0.131060524932472
776 真値: [2.71023895 4.40749862 3.31122278] 推定値 [ 4.0780821  -6.26590039  8.00753948] 誤差: 10.76068962213892
777 真値: [-4.16865918  1.25943636  7.77789949] 推定値 [-3.50361355  2.45809948  3.9732527 ] 誤差: 1.3707950144661094
778 真値: [-0.04151268  2.65028335  3.26980852] 推定値 [0.12398164 2.49697779 9.56943979] 誤差: 0.22559025645286035
779 真値: [-0.53

845 真値: [-2.10572328 -3.70048081  5.19546751] 推定値 [-0.79316404 -2.67671704  5.38554539] 誤差: 1.6646032579251913
846 真値: [ 3.39024926 -4.51906571  7.46173658] 推定値 [-0.43157928 -2.51992312  4.74284963] 誤差: 4.313113086623496
847 真値: [4.12338542 1.73978458 1.88512971] 推定値 [ 3.17079928  0.91109942 -5.06137684] 誤差: 1.2625923554086849
848 真値: [-0.57186179  2.13307622  5.30055213] 推定値 [-0.54643639  3.25970877  5.24646198] 誤差: 1.1269194129248492
849 真値: [4.69344633 2.87831376 3.90883449] 推定値 [4.71680423 3.08921533 4.02491331] 誤差: 0.21219109974215644
850 真値: [-4.48958263 -1.18104973  8.09768738] 推定値 [-0.92385354  1.43731942  3.32186455] 誤差: 4.423831030594297
851 真値: [-0.93647754 -2.25906555  7.72907125] 推定値 [0.26112138 0.05150602 1.90100456] 誤差: 2.6024957575356695
852 真値: [-0.85775083  1.21897187  1.97012419] 推定値 [-0.21915752 -0.07212857  1.69427178] 誤差: 1.4403963919928582
853 真値: [-3.81207503  1.3922541   3.95803038] 推定値 [-3.89894145  1.29533738 10.20392019] 誤差: 0.1301484728081706
854 真値: [0.025

921 真値: [-5.93371946  1.93516804  5.69622736] 推定値 [-2.22378884  2.66122393  9.04564022] 誤差: 3.7803098246851046
922 真値: [-0.97907128  4.94260218  8.68861625] 推定値 [-1.38785028  4.1175243   8.78115436] 誤差: 0.9207897567028226
923 真値: [-4.72553152 -2.21798704  7.04857367] 推定値 [-0.17947751 -0.79015685  8.12305774] 誤差: 4.76500851929078
924 真値: [ 1.70269683 -0.61709266  3.02945177] 推定値 [ 1.98345563 -0.15985197  9.31505156] 誤差: 0.5365580581133347
925 真値: [2.94533342 1.66781096 3.86308503] 推定値 [2.77817521 1.65412908 3.90590404] 誤差: 0.16771721131137307
926 真値: [-3.94906723  2.27817962  4.37432323] 推定値 [-3.56850743  2.07520052  3.20911092] 誤差: 0.43130763312203146
927 真値: [ 5.97955626 -4.82787394  7.68865341] 推定値 [-0.23338734  0.40965087  4.40648238] 誤差: 8.12602819792859
928 真値: [1.96351571 2.12191091 3.46812288] 推定値 [1.66779982 2.41877235 3.40855852] 誤差: 0.41901622871858785
929 真値: [ 3.52360898 -0.03514144  5.73063801] 推定値 [3.99285808 0.12358718 5.59991513] 誤差: 0.49536803448944855
930 真値: [-2.7581

996 真値: [ 3.92360641 -4.64676812  8.2484732 ] 推定値 [-0.66991577 -1.2475621   5.2353149 ] 誤差: 5.714459512685757
997 真値: [4.17237882 4.63103697 3.12369202] 推定値 [1.06015718 3.90688024 4.88158679] 誤差: 3.1953601590680587
998 真値: [5.93792003 0.60539112 1.95388989] 推定値 [ 0.97259146 -0.52249409  0.52951691] 誤差: 5.091818226295429
999 真値: [0.06253807 0.38398216 2.89497825] 推定値 [-0.01032216  1.19157189  3.00538114] 誤差: 0.8108697704542833


505